In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 4.2.- Juego Space Invaders de Open AI GYM (con Keras RL)


* GYM de Open AI https://gym.openai.com/: Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de Aprendizaje por Refuerzo (RL). Esta librería ofrece diferentes juegos (entornos) como los juegos de Atari en los que poder probar los algoritmos de RL.


* Keras RL https://keras-rl.readthedocs.io/: keras-rl implementa algunos algoritmos de Aprendizaje por Refuerzo en Python y se integra con con Tensorflow-Keras. Además, keras-rl esta listo para su uso con OpenAI Gym, por lo que resulta sencillo evaluar mediante los juegos de GYM el rendimiento de los Algoritmos.

<hr>

## 1.- Instalación de las librerías (en un Python 3.7)

In [12]:
!pip3 install tensorflow==2.6.3 --user
!pip3 install keras-rl2 --user
!pip3 install atari_py --user
!pip3 install gym[atari,accept-rom-license]==0.23.0 --user

  Using cached gym-0.23.0-py3-none-any.whl


<hr>

## 2.- Importamos la librería de GYM y atari

In [13]:
import atari_py
import gym

# Definición de constantes
ENVIRONMENT_NAME = 'SpaceInvaders-v0'
FILE_MODEL_NAME = "Keras_SpaceInvaders_Model"

<hr>

## 3.- Instanciamos el entorno del Space Invaders y jugamos con un agente que toma acciones aleatorias

In [38]:
# Instanciamos el Entorno
env = gym.make(ENVIRONMENT_NAME)

# Obtenemos el tamaño de las imagenes del juego y los canales
height, width, n_channels = env.observation_space.shape
print('Tamaño de la imagen: {}x{} - Nº Canales: {}'.format(width, height, n_channels))

# Vemos las posibles acciones del juego:
n_actions = env.action_space.n
actions_info = env.unwrapped.get_action_meanings()
print('\nNº de acciones: {} - Acciones: {}\n'.format(n_actions, actions_info))

# Jugamos 5 partidas de manera aleatoria
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()  # Inicializamos el entorno
    done = False         # Flag de finalización de la partida (episodio)
    score = 0            # Contador de recompensas
    
    # ! A Jugar ¡ (Hasta que termine la partida -> done == True)
    while not done:
        env.render(mode='human')
        action = env.action_space.sample()              # Seleccionamos una acción Aleatoria
        n_state, reward, done, info = env.step(action)  # Realizamos la acción aleatoria y obtenemos: 
                                                        # 1. Lista de estados, 2. Recompensa, 3. ¿Fin del juego?, 4. info
        score+=reward                                   # Sumamos la recompensa de la acción
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


Tamaño de la imagen: 160x210 - Nº Canales: 3

Nº de acciones: 6 - Acciones: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

Episode:1 Score:80.0
Episode:2 Score:125.0
Episode:3 Score:45.0
Episode:4 Score:210.0
Episode:5 Score:35.0


<hr>

## 4.- Creamos un modelo de red neuronal convolucional


* Para tomar las acciones vamos a usar una red neuronal convolucional que:

    + Tomará como entrada la imagen de la partida (Alto de 210px, Ancho de 160px y 3 canales)
    + La capa de salida tendrá 6 Neuronas, 1 por posible acción

In [44]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Convolution2D(filters=32, 
                        kernel_size=(8,8), 
                        strides=(4,4), 
                        activation='relu', 
                        input_shape=(3, height, width, channels)))
model.add(Convolution2D(filters=64, 
                        kernel_size=(4,4), 
                        strides=(2,2), 
                        activation='relu'))
# model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(n_actions, activation='linear'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
flatten_3 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               42467840  
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 1542      
Total params: 42,639,718
Trainable params: 42,639,718
Non-trainable params: 0
__________________________________________

<hr>

## 5.- Construimos el agente con Keral RL

In [45]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy


memory = SequentialMemory(limit=1000, window_length=3)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
dqn = DQNAgent(model=model, 
               memory=memory, 
               policy=policy,
               enable_dueling_network=True, 
               nb_actions=actions, 
               nb_steps_warmup=1000)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=1000, visualize=False, verbose=2)

Training for 1000 steps ...
 556/1000: episode: 1, duration: 16.076s, episode steps: 556, steps per second:  35, episode reward: 105.000, mean reward:  0.189 [ 0.000, 30.000], mean action: 2.394 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 944/1000: episode: 2, duration: 11.072s, episode steps: 388, steps per second:  35, episode reward: 55.000, mean reward:  0.142 [ 0.000, 20.000], mean action: 2.621 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
done, took 28.779 seconds


In [46]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 155.000, steps: 986
Episode 2: reward: 120.000, steps: 682
Episode 3: reward: 85.000, steps: 630
Episode 4: reward: 85.000, steps: 919
Episode 5: reward: 125.000, steps: 701
Episode 6: reward: 55.000, steps: 507
Episode 7: reward: 200.000, steps: 961
Episode 8: reward: 5.000, steps: 388
Episode 9: reward: 30.000, steps: 610
Episode 10: reward: 30.000, steps: 513
89.0


<hr>

## 6.- Exportamos el agente entrenado

<hr>

## 7.- Importamos el modelo (agente) y Jugamos